In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from mp.Dataprocessor import *
from mp.Modelevaluator import *
from sklearn.preprocessing import Normalizer
import os
os.chdir(os.getcwd()+'\\data')


In [2]:
dl=Data_Loader()
data=dl.upload()

dp=Data_Processor
d_train, d_test = dp.test_train_split(data)
x_train,y_train,x_test,y_test = dp.normaliser(d_train,d_test)

In [3]:
#Cross Validate Logistic Regression Classifier
rf = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=5, max_depth=5)
mod = Modelevaluator(rf,x_train, x_test, y_train, y_test)

print(mod.cross_evaluation(cv_int=5))
print(mod.r2score())

Overfitting , mean_train_score: 0.986 , mean_test_score: -4.973
None
0.9854502529166244


In [4]:
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
print(os.getcwd())

C:\Users\anirban\Desktop\test_ml


In [5]:
mod.save_model('xgbregressor.pkl')

In [6]:
d_test_norm=pd.DataFrame(Normalizer().fit_transform(d_test.loc[:,'sens_1':'sens_5']))
cols=list(d_test.columns)
d_test_norm.columns=cols
d_test_norm['sens_1_pred']=mod.generate_predictions()
d_test_norm=d_test_norm.set_index(d_test.index)

In [7]:
d_test_norm.to_csv('test_data.csv',index=True)

In [13]:
from sqlalchemy import *
from sqlalchemy import create_engine
from sqlalchemy import text

engine = create_engine('sqlite:///pyakc.db', echo=False)
qry_table=pd.read_sql_query("SELECT * FROM sensor", con=engine)
qry_table.head(10)

,index,created_timestamp,sens_1,sens_2,sens_4,sens_5,sens_1_pred
0,0,2021-07-05 17:15:55.968000+00:00,0.662178,0.747470,0.018609,0.049625,0.662820
1,1,2021-07-05 17:15:59.079000+00:00,0.658673,0.750545,0.018686,0.049829,0.659952
2,2,2021-07-05 17:16:02.142000+00:00,0.661411,0.748153,0.018588,0.049567,0.659952
3,3,2021-07-05 17:16:06.736000+00:00,0.657904,0.751224,0.018664,0.049771,0.658106
4,4,2021-07-05 17:16:12.974000+00:00,0.658896,0.750366,0.018604,0.049611,0.659952
5,5,2021-07-05 17:16:16.144000+00:00,0.660645,0.748834,0.018566,0.049510,0.659952
6,6,2021-07-05 17:16:19.314000+00:00,0.659880,0.749512,0.018545,0.049452,0.659952
7,7,2021-07-05 17:16:20.952000+00:00,0.658130,0.751042,0.018582,0.049553,0.659952
8,8,2021-07-05 17:16:24.033000+00:00,0.661620,0.747985,0.018507,0.049352,0.659952
9,9,2021-07-05 17:16:27.207000+00:00,0.656369,0.752574,0.018620,0.049654,0.656174


In [12]:
#engine.execute('drop table sensor')
